In [1]:


import pandas as pd
import numpy as np
import gzip
import os

from datetime import timedelta 
import pandas as pd
import calendar
import numpy as np
import glob
import sys
import datetime 

import re
import os
import quantstats as qs
import time

import matplotlib.pyplot as plt

import nest_asyncio # install this package to avoid running in loop

import urllib

import pandas_market_calendars as mcal

import pandas_ta as pta
import asyncio
import datetime
qs.extend_pandas()

# import dask.dataframe as dd
# from dask.distributed import Client
from concurrent.futures import ThreadPoolExecutor

#import talib


import sys
#sys.path.append(r'C:\Users\Administrator\Desktop\Indian_backtester_python')


#from fetchHistData import *

from strategy_functions import *   # Import the module
from live_functions import *

from more_functions import*



#==== expiry dates files path
stock_Exp = r"/home/aakash_tandon/Python_strategies/India_stock_futures_expiries.csv"
bk_exp = r"/home/aakash_tandon/Python_strategies/Banknifty_weekly_expiries.csv"
nif_exp = r"/home/aakash_tandon/Python_strategies/nifty_weekly_expiries.csv"
# mid_exp
fin_exp = r"/home/aakash_tandon/Python_strategies/Finnifty_weekly_expiries.csv"
# sensex_exp



#== read_expiry files
stock_expiry = pd.read_csv(stock_Exp )
nifty_weekly_exp = pd.read_csv(nif_exp)
bank_weekly_exp = pd.read_csv(bk_exp)
fin_weekly_exp = pd.read_csv(fin_exp)


strategy_name = "Live_aakash_fno_CF_v1"


df_folder , trades_strategy_folder , position_folder , strategy_folder = initialise_strategy_directory(strategy_name=strategy_name , root_folder=r"/home/aakash_tandon/Python_strategies")


#=== one is to generate trade statistics and another to store all trades

output_all_trades_file = os.path.join(strategy_folder, f"All_trades_{strategy_name}.csv")
trade_html_file = os.path.join(strategy_folder, f"Trade_stats_{strategy_name}.html")



#=== process the historical and current expiry dates for any asset
nifty_weekly_exp = process_expiry_dates_from_file(nifty_weekly_exp)




In [2]:
trades_strategy_folder

'/home/aakash_tandon/Python_strategies/Live_aakash_fno_CF_v1/symbol_wise_trades'

In [3]:
fetch_live_candles(symbol='NSE:ACC' , lookback_days=0 , product='stocks' , interval=30)

Data is empty.


In [4]:

def process_index(ind_df):
    
                 
    ind_df = ind_df[(ind_df.index.dayofweek<5)]
     
    
    #print(ind_df) 
    ind_df['ind20DMA'] = daily_moving_average(df=ind_df, timeframe='D' , column='indHigh', periods=3, agg_func='last')
        
    ind_df['indtodayo'] = ind_df.indOpen.groupby(ind_df.index.date).first().reindex(ind_df.index.date).values
    ind_df['indprevdayc'] = ind_df.indClose.groupby(ind_df.index.date).last().shift().reindex(ind_df.index.date).values
    ind_df['indprevdayh'] = ind_df.indHigh.groupby(ind_df.index.date).max().shift().reindex(ind_df.index.date).values
    ind_df['indprevdayo'] = ind_df.indOpen.groupby(ind_df.index.date).first().shift().reindex(ind_df.index.date).values
                                                      
    ind_df['near_Exp'] = get_prev_exp(ind_df.index , symbol='NIFTY' ,expirylist=nifty_weekly_exp)
    
    ind_df['ind_todayh'] = intraday_high(df=ind_df ,high_col= 'indHigh' ,agg_func= 'max')
    ind_df['ind_todayl'] = intraday_low(df=ind_df ,low_col= 'indLow' , agg_func='min')
            
    ind_df['indexpiry_high'] = expanding_high_since_expiry(ind_df , expiry_series='near_Exp' , column='indHigh')
        
    ind_df['indexpiry_low'] = expanding_low_since_expiry(ind_df , expiry_series='near_Exp' , column='indLow')
            
    ind_df = ind_df[(ind_df.index.year>=2023) ]
        
    #ind_df = ind_df.reindex(combined_df.index, method='ffill')
            
    return ind_df


def fetch_and_rank_data(symbol, lookback, time_interval=30, index_req=1, indexdf=None):
    
    """
    Fetches historical data for a given symbol and ranks it based on a parameter.
    
    Parameters:
    - symbol (str): The symbol for which to fetch and rank data.
    - lookback : days to which fetch hisorical data from yesterday
    - time_interval (int): Time interval for the data (default 30).
    - index_req (int): Whether index data is required for ranking (1 = yes, 0 = no).
    - indexdf (pd.DataFrame): DataFrame containing index data for ranking.
    
    Returns:
    - pd.DataFrame: DataFrame containing the ranked data or None if data is empty or an error occurs.
    """
    encoded_symbol = urllib.parse.quote(symbol)
    
    end_date = datetime.today().date()
    start_date = end_date - timedelta(days=lookback)
    
    
    start_date_str = start_date.strftime('%Y-%m-%d')
    end_date_str = end_date.strftime('%Y-%m-%d')
    
    
    url = f"http://65.0.126.220:8002/api/stocks/candles?interval={time_interval}&instrument={encoded_symbol}&from={start_date_str}&to={end_date_str}"
    
    
    print("\n Processing symbol  " , symbol)

    try:
        response = requests.get(url)
        response.raise_for_status()
        data_json = response.json()
    except requests.RequestException as e:
        print(f"Request failed: {e}")
        return None
    
    if not data_json.get('data'):
        print("\n Data is empty for symbol" , symbol )
        return None
    
    df = pd.DataFrame(data_json['data'])
    if 'Timestamp' in df.columns:
        df['Timestamp'] = pd.to_datetime(df['Timestamp'])
        df.set_index('Timestamp', inplace=True)
    
    else:
        print("\n No timestam found in symbol " , symbol , "\n Data is as " , df )
        
    #print("\n \n Dataframe is :" , df)
    #df['Symbol'] = symbol  # Add symbol to the DataFrame
    
    # Continue to ranking if data is fetched successfully
    if index_req == 1 and indexdf is not None:
        
        #print("\n Index dataframe is : " , indexdf)
        indexdf = indexdf.reindex(df.index, method='ffill')
        df = pd.concat([df, indexdf], axis=1)

        #print("\n Dataframe after index" , df)
    #sym = df['Symbol'].iloc[0]
    #print("Processing for rank:", symbol)
    
    beta = calculate_rolling_beta_from_intraday(df, window=5, stock_col='Close', index_col='indClose')
    df['rank_param'] = beta
    
    #print(df['rank_param'])
    
    columns_mapping = {'rank_param': f'{symbol}_rank_param'}
    df_renamed = df.rename(columns=columns_mapping)
    df_renamed = df_renamed[f'{symbol}_rank_param']
    
    return df_renamed if not df_renamed.empty else None





def process_index_historial(index_symbol='NSE:NIFTY 50' , lookback_days=40 , bar_interval=30):
    
        
    end_date = datetime.date.today()
    start_date = end_date - timedelta(days=lookback_days)
    
        
    ind_df = fetch_historical_index_from_date(symbol=index_symbol , time_interval=bar_interval , start_date=start_date , end_date=end_date)

    if ind_df is None:
        print("No data returned.")
    
        
        
    #ind_df = process_index(ind_df )
    
    ind_df = ind_df.rename(columns={'Open': 'indOpen','High': 'indHigh', 'Low': 'indLow', 'Close': 'indClose', 'Volume':'indVolume', 'Symbol': 'indSymbol'})
    
    return ind_df    
        
    


def fetch_historical_data_from_date(symbol, time_interval=30, start_date=datetime.date(2023 , 9 , 1 ,) , end_date=datetime.date(2024 , 3 , 1) ):
    """
    Fetch historical data for a given symbol from the specified URL.
    
    Parameters:
    - symbol (str): The symbol for which to fetch data.
    - time_interval (int): The time interval for the data.
    - bar_count (int): The number of data points to fetch.
    
    Returns:
    - pd.DataFrame: DataFrame containing the historical data or None if data is empty.
    """
    # URL encode the symbol to handle spaces and special characters
    encoded_symbol = urllib.parse.quote(symbol)
    
    # Convert dates to strings in the format 'YYYY-MM-DD'
    start_date_str = start_date.strftime('%Y-%m-%d')
    end_date_str = end_date.strftime('%Y-%m-%d')
    
    
    # Construct the URL with the given parameters
    url = f"http://65.0.126.220:8002/api/stocks/candles?interval={time_interval}&instrument={encoded_symbol}&from={start_date_str}&to={end_date_str}"

    # Send GET request
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raises an HTTPError if the response status code was unsuccessful
    except requests.RequestException as e:
        print(f"Request failed: {e}")
        return None
    
    # Convert response to JSON
    data_json = response.json()
    
    # Check if data is empty
    if not data_json.get('data'):
        print("Data is empty.")
        return None
    
    # Create DataFrame from data
    df = pd.DataFrame(data_json['data'])
    
    # Convert 'Timestamp' to datetime and set as index
    df['Timestamp'] = pd.to_datetime(df['Timestamp'])
    df.set_index('Timestamp', inplace=True)
    
    return df




In [3]:



def process_symbol2(df , index):
    
    
    sym = df['Symbol'].iloc[0]
    if df.empty:
        print("No data fetched.")
        return
    else:
        print("\Processing symbol " , sym)
    
    if not isinstance(df.index, pd.DatetimeIndex):
        print("Error: DataFrame index is not of type DatetimeIndex.")
        return None, "Error"
    
    
    #df['first_hr_high'] =  x_min_high(df , 'High' , start='09:30' , end='11:00' )
   
    df['x_day_low'] = get_x_day_low(df, n=10 , column='Close')
    df['x_day_high'] = get_x_day_high(df , n=20 , column='High')
    #df['m_low'] = get_x_day_low(df , n=40 , column='Close')
    #df['low_touch'] = np.where(df['Close']<df['x_day_low'].shift() , 1 , 0)
    
    #df[fname+'_ratio'] = df['cval']/df['D_Volume_moving_average']
    #df['bars_since_low'] = bars_since_condition(df , column='low_touch')
        
    df['5drng'] = get_x_day_range(df , n=5, high_col='High' , low_col='Low')
            
    df['todayo'] = df.Open.groupby(df.index.date).first().reindex(df.index.date).values
    df['prevdayc'] = df.Close.groupby(df.index.date).last().shift().reindex(df.index.date).values
    #df['ratio_co'] = df['Close']/(df[['todayo', 'prevdayc']].max(axis=1))
    df['prevdayl'] = df.Low.groupby(df.index.date).min().shift().reindex(df.index.date).values
    df['prevdayh'] = df.High.groupby(df.index.date).max().shift().reindex(df.index.date).values
    df['prevdayo'] = df.Open.groupby(df.index.date).first().shift().reindex(df.index.date).values
    #df['slope'] = find_slope_and_prediction(df , 'Low' , window=100)[0]
    df['vwap'] = intraday_vwap(df)
    
    
    df = pivot_points_classic(df)
    
    df['todayl'] = intraday_low(df=df , low_col='Low' , agg_func='min')
    df['todayh'] = intraday_high(df=df , high_col='High' , agg_func='max')
    df['daily_RSI'] = rsi_timeframe(df ,period=7 , timeframe='1D' )
    df['zc_ROC'] = daily_ROC(df , column='Close' , ROC_period=3 , z_score_period=40)
    #df = df[(df.index.year>=2023)]
    #print("\n" , df.columns)
    
    #====== Index based conditions in any
        
    #===== First reindex the index to df
    index = index.reindex(df.index, method='ffill')
    
    df = pd.concat([df, index], axis=1)
                
                    
    #print(  "\n Concated df with index is : " , df)
    
    #df['int_indHigh'] = intraday_high(df=df , high_col='indHigh' , agg_func=)
    
    #=== Long conditions
       
    condition1 = df.index.time>time(10 , 15) 
    condition2 = df["indprevdayc"]>df["indprevdayo"]*0.985    #df['Close']>df['x_day_high'] - (0.5*(df['x_day_high']-df['x_day_low']))
    
    #condition4 = df['ATR_zc']<=-1
    #condition5 = df['indLow']<df['ind20DMA']
    condition6 = df['prevdayc']>df['prevdayo']*0.98
    #condition7 = df['todayo']>df['m_low']*1.03
    condition8 = df['Close']>df['vwap']*0.995
    condition9 = df['Close']>=df['prevdayl']*1.001
    condition10 = df['daily_RSI']<80
    condition11 = df['todayo']>df['S2']
    condition12 = df['indtodayo']>df['indprevdayc']*0.998  # much better than opposite
    condition13 = df['indClose']>df['ind_todayl']*1.003
    #condition14 = df['ATR_zc']>=-1
    #condition15 = df['todayo']<df['prevdayh']*1.02
    condition17 = df['prevdayc']<df['prevdayo']*1.05
    condition18 = df['zc_ROC']<2.25
 
    
    #=== price goes below S2 but now above S1
    df['S1_reclaim'] = np.where(((df['todayl']<df['R1']*1.002) & (df['Close']>df['R1'].shift()))  , 1 , 0)
    condition16 = df['S1_reclaim']>=1
    
    ofol = df_folder
    ofil = os.path.join(ofol , f'{sym}.csv')
    #print(ofil)
    df.to_csv(ofil)
    
    #df['prevdayc']<df['prevdayo']*1.02 not helping
    #condition14 = df['todayl']>df["last_trough"]*1.01
    #condition14 = df['Close']<df['R2']
    
    #& condition13 is gud
    
    #=== df['Close']>df['first_hr_high'].shift(1) not helping
    #== df['Close']>df['x_day_high'] - (0.25*(df['x_day_high']-df['x_day_low'])) not helping
    #=== 20 dma index MA not helping
    
    # & condition10 trying.. not thta help
    #=== short conditions
    
    # & condition3
    # & condition6
    # & condition14 - df['slope']>0 not helping
    # ATR_zc bad
    # & condition14  & condition10  & condition12  & condition5 & condition7  & condition11
    
    # & condition17
    #& condition15 slight gud
    df['signal'] = condition1 & condition2 & condition12 & condition16 & condition13 & condition18 & condition17 
        
    #print("\n Mean of symbol is " , df['Close'].mean())
    
    # Compute the final result and return it as a pandas DataFrame
    return df 
    


In [17]:

position = {}
def process_signal(df, side=1 , intra_Exit=0 , n_day_Exit=2 ):
    
    #================================================================================================
    
    symbol = df['Symbol'].iloc[0]
    
    if symbol is None:
        raise ValueError("Symbol not in columns")
    
    print("\n Processing signals of : ", symbol)
    log = []
    bar_counts = [0] * len(df)
    positions = [0]* len(df)
    position = 0
    day_count = 0
    entry_price = 0 
    exit_price = 0 
    ordern = 0 
    orders = [0]*len(df)
    highest_high = 0
    #position[symbol]['position'] = 1
    
    #print(position)
    
     
    # Compute date changes using diff()
    #df['date_changed'] = df.index.to_series().diff().dt.days != 0
    
        # Ensure your index is a DateTimeIndex and sorted.
    df['date_changed'] = df.index.to_series().dt.date != df.index.to_series().shift(1).dt.date
    
    #================================================================================================
               
    for i in range(len(df)):
        
        row = df.iloc[i]
        current_date = df.index[i].date()
                
        # If in a position, increment the bar count
        if position != 0:
                        
            #df.loc[i, 'bar_count'] = df.loc[i - 1, 'bar_count'] + 1  # increment bar_count in df
            bar_counts[i] = bar_counts[i-1] + 1
            
            if row['date_changed']:
                #print("the date changed")
                day_count += 1
                                           
            else:
                day_count = day_count
        else:
            #bar_count = 0
            #df.loc[i, 'bar_count'] = 0 
            bar_counts[i] =0 
            ordern = 0 
            day_count = 0
            entry_price = 0
        
        #--- n-day exit at first bar....
        
        highest_high = row['High'] if position > 0 and row['High'] > highest_high else highest_high if position > 0 else 0
        lowest_along = row['Low'] if position > 0 and row['Low'] < lowest_along else lowest_along if position > 0 else 0
        
        
        if ( df['signal'][i-1]==1 or df['signal'][i]==1) and position == 0  and df['Close'][i]<df['Close'][i-1] and df['indClose'][i]>df['indtodayo'][i]*0.999 :
            # Determine if it's a long or short entry
            if side > 0:
                position = 1
                entry_price = row['Close']
                ordern = 1
                bar_counts[i]+=1                 
                print("\n Position taken long at : " , df.index[i] , "-- entry Price - :  " , entry_price , 'R1 today' ,df['R1'][i]*1.002  )
                log.append((df.index[i], symbol, position , entry_price , ordern))
                #print("\n" , df['5drng'][i])
            else:
                position = -1
                entry_price = row['Close']
                ordern = -1 
                #print("\n Position taken Short at : " , df.index[i] , "-- entry Price - :  " , entry_price)
                log.append((df.index[i], symbol, position , entry_price, ordern))
        
        #print("\n Index" , df.index[i] ,  ":  position: " , position , ": highest_high : " , highest_high  , "bar_count :" , bar_counts[i] , "Close: " , df['Close'][i]  )
        
        if (position == 1 and row['Close'] < min(df['prevdayl'][i]*0.995 ,df['todayl'][i-1]*1.005) and bar_counts[i]>=2):
            
            print("\n Position Hit 2 day low " , df.index[i] , position , "position at  ", row['Close'] )
            position = 0
            #bar_counts[i] = 0
            exit_price = row['Close']
            ordern = 13
            log.append((df.index[i], symbol, position , exit_price, ordern))

        
        
        # End of Day Exit condition based on parameter intra_exit
        
        #print("position " ,position , df['bar_count'][i])
        
        if position != 0 and intra_Exit==1 and df.index[i].time() >= datetime.time(15, 5) and bar_counts[i] > 1:
            
            #print("Hi")
            #print("\n Position exit EOD long at" , df.index[i] , position , "position at  ", row['Close'] )
            
            position = 0
            #bar_counts[i] = 0
            exit_price = row['Close']
            ordern = 11 
            log.append((df.index[i], symbol, position , exit_price , ordern))
        
        
        if position != 0 and bar_counts[i]>4 and df['indClose'][i]<df["indOpen"][i-1]*0.985 and df['High'][i]<df['todayh'][i-1]*0.985:
            
            #print("Hi")
            print("\n Nifty bear exit at: " , df.index[i] ,"Nifty high and highest are:  "  ,df['indHigh'][i] , df['ind_todayh'][i-1]  )
            
            position = 0
            #bar_counts[i] = 0
            exit_price = row['Close']
            ordern = 12 
            log.append((df.index[i], symbol, position , exit_price , ordern))
                       
                
        if position > 0 and day_count>=1 and bar_counts[i]>6 and df['Close'][i]<df['prevdayl'][i]*0.97 and df['Close'][i]<df['Close'][i-1] :
            print("\n Gap down exit EOD long at" , df.index[i] , position , "position at  ", row['Close'] , "Day_count: " , day_count , "close: " , df['Close'][i] , df['Close'][i-1] )
        
            position = 0
            #bar_counts[i] = 0
            exit_price = row['Close']
            ordern = 14
            log.append((df.index[i], symbol, position , exit_price , ordern))
        
        if position > 0 and day_count>=1 and bar_counts[i]>3 and df['Close'][i]<df['Low'][i-1]*0.985  :
            print("\n Large bear bar" , df.index[i]  , "Day_count: " , day_count , "close: " , df['Close'][i] , df['Open'][i] )
        
            position = 0
            #bar_counts[i] = 0
            exit_price = row['Close']
            ordern = 15
            log.append((df.index[i], symbol, position , row['Close'], ordern))
                
        
        if position != 0 and intra_Exit==0 and day_count>=n_day_Exit and bar_counts[i]>1 and df['Close'][i]<df['Close'][i-1]  :
            
            print("\n Carryforward exit EOD long at" , df.index[i] , position , "position at  ", row['Close'] , "Day_count: " , day_count  )
            
            position = 0
            #bar_counts[i] = 0
            exit_price = row['Close']
            ordern = 16
            log.append((df.index[i], symbol, position , row['Close'], ordern))
                
        
        #---- Stop-loss of 3%
        
        if position!=0 and df['todayo'][i]<df['x_day_low'][i] and df['Close'][i]<df['Low'][i-1] and day_count>=1 and df.index[i].time()>=datetime.time(10 , 45):
            
            position = 0
            #bar_counts[i] = 0
            exit_price = row['Close']
            ordern = 17
            log.append((df.index[i], symbol, position , row['Close'], ordern))
            
            print("\nBreakdown opening exit from low:  " , df['x_day_low'][i] ,   " -- open at" , df['todayo'][i] , "exit at : " , exit_price , "time is " , datetime.time(10 , 45) )
        
        if position!=0 and df['High'][i]<df['S1'][i-1] and df['Close'][i]<df['Close'][i-1] and day_count>=1 and df.index[i].time()>=time(10 , 45):
            
            position = 0
            #bar_counts[i] = 0
            exit_price = row['Close']
            ordern = 18
            log.append((df.index[i], symbol, position , row['Close'], ordern))
            print("\S1 broken down exit:  S1 is : " , df['S1'][i-1] ,   " -- exit  at" , exit_price ,  "time is:  " , df.index[i].time()  )
        
        
        #=== Fixed 2% stop loss
        if position!=0 and df['Low'][i]<min(df['Open'][i] , entry_price*0.98) and bar_counts[i]>1:
            
            
            position = 0
            #bar_counts[i] = 0
            exit_price = min(df['Open'][i] , entry_price*0.98)
            ordern = 19
            log.append((df.index[i], symbol, position , exit_price, ordern))

            print("\ Fixed stoploss:  S1 is : " , min(df['Open'][i] , entry_price*0.98) ,   " -- exit  at" , exit_price ,  "time is " , df.index[i].time() , "Low of bar is :" , df['Low'][i] )
            
              
    
        #----- 5drnag exit
        
        if position == 1 and row['Close'] > entry_price*(1+(1.25*df['5drng'][i])) and df['Close'][i]>df['Close'][i-1] and bar_counts[i]>10:
            
            print("\n Range target on " , df.index[i] , "price at :   ", row['Close'] , " range: " , df['5drng'][i] , "entryprice" , entry_price )
            position = 0
            #bar_counts[i] = 0
            ordern = 19
            exit_price = row['Close']
            log.append((df.index[i], symbol, position , exit_price, ordern))
        
         #----- reistance exit
        
        if position >0 and df['Close'][i-1]>df['R2'][i-1]*1.01 and df['Close'][i]<df['R2'][i-1]*1.01 and bar_counts[i]>5 and df.index[i].time() >= time(9, 30):
            
            print("\n Resistance target  " , df.index[i] , "at  ", row['Close'] , " R2 prev: " , df['R2'][i-1] , " entryprice" , entry_price )
            position = 0
            #bar_counts[i] = 0
            exit_price = row['Close']
            ordern = 20
            log.append((df.index[i], symbol, position , exit_price, ordern))
        
        positions[i] = position
        orders[i] = ordern
   
    
    #=== outside for loop 
    
    #print("\n Position at last bar is " , positions[-1]   )
    print(log)
    
    df[symbol+'bar_count'] = bar_counts
    df[symbol+'_order'] = orders
    df[symbol+'_position'] = positions
    
    #== these are bar-by-bar returns when you are in position 
    df[symbol+'_ret'] = np.where(df[symbol+'bar_count']>0 ,  df['Close'].pct_change()*side , 0)
    
    
    #=== 
    df[symbol+'_Close'] = df['Close']

    
    #print(df)
    
    trade_log = create_tradelog2(df , symbol)


#     if( num_entries!=num_exits):

#         print("\n \n Number of entries and exits dont match"  , "Entries: " , num_entries , "----- Exits" , num_exits  )

    file_path = os.path.join(trades_strategy_folder, f"{symbol}.csv")
    
    print("\n \n The Outputting to trade file " , file_path)
    
    trade_log.to_csv(file_path )

    
    ofol = df_folder
    ofil = os.path.join(ofol , f'{symbol}.csv')
    #print(ofil)
    df.to_csv(ofil)
    
    print("\n \n The Outputting dataframe to file file " , ofil)
    
        
    
    return df[[symbol+'_Close' , symbol+'_position' ,symbol+'_ret' , symbol+'bar_count', symbol+'_order']] 


In [5]:
import aiohttp
import asyncio
import pandas as pd
from datetime import datetime, timedelta
import urllib.parse

async def fetch_historical_data_from_date_async(symbol, time_interval=30, start_date=datetime(2023, 9, 1), end_date=datetime(2024, 3, 1)):
    encoded_symbol = urllib.parse.quote(symbol)
    
    start_date_str = start_date.strftime('%Y-%m-%d')
    end_date_str = end_date.strftime('%Y-%m-%d')
    url = f"http://65.0.126.220:8002/api/stocks/candles?interval={time_interval}&instrument={encoded_symbol}&from={start_date_str}&to={end_date_str}"

    async with aiohttp.ClientSession() as session:
        try:
            async with session.get(url) as response:
                response.raise_for_status()
                data_json = await response.json()
        except aiohttp.ClientError as e:
            print(f"Request failed for {symbol}: {e}")
            return pd.DataFrame()

    if not data_json.get('data'):
        print(f"No data found for {symbol}.")
        return pd.DataFrame()

    df = pd.DataFrame(data_json['data'])
    df['Timestamp'] = pd.to_datetime(df['Timestamp'])
    df.set_index('Timestamp', inplace=True)
    
    return df



async def fetch_historical_data_from_date_for_symbols(symbols, time_interval=30, start_date=datetime(2023, 9, 1), end_date=datetime(2024, 3, 1)):
    async with aiohttp.ClientSession() as session:
        tasks = {
            symbol: fetch_historical_data_from_date_async( symbol, time_interval, start_date, end_date)
            for symbol in symbols
        }
        # Use asyncio.gather to fetch data concurrently, and unpack the dictionary values as arguments
        results = await asyncio.gather(*tasks.values())
        # Combine the tasks' keys (symbols) with their corresponding results (DataFrames) into a dictionary
        return {symbol: result for symbol, result in zip(tasks.keys(), results)}





In [ ]:
df = await fetch_historical_data_from_date_async(symbol='NSE:ACC' , time_interval=30 , start_date=datetime.today().date() - timedelta(days=30) , end_date=datetime.today().date()-timedelta(days=1))

In [ ]:
all_data = await fetch_historical_data_from_date_for_symbols(symbols=today_stocks , time_interval=30 , start_date=datetime.today().date() - timedelta(days=30) , end_date=datetime.today().date()-timedelta(days=1))

In [ ]:
all_data = await fetch_historical_data_from_date_for_symbols(symbols=today_stocks , time_interval=30 , start_date=datetime.today().date() - timedelta(days=30) , end_date=datetime.today().date()-timedelta(days=1))

In [6]:
def pre_strategy_code(stock_limit , lookback):
    # Your code that runs before the strategy starts everyday
    
    print("Running pre-strategy code...")
    
    fno_stocks = get_fno_stocks()
    
    
    stocks_remove = ['ABFRL', 'ATUL', 'MIDCPNIFTY', 'COROMANDEL', 'CROMPTON' , 'DABUR' , 'GMRINFRA' , 'IDEA' , "ICICIGI" , 'RAMCOCEM' , 'AARTIIND' , 'ZEEL', 'NIFTY' , 'BANKNIFTY' , 'FINNIFTY' ]


    
    stocks_to_trade = [stock for stock in fno_stocks if stock not in stocks_remove]
    
    
    stocks_to_process = ['NSE:'+symbol for symbol in stocks_to_trade]
    print("Total stocks to consider" , len(stocks_to_process))
    
    
    #== now we get historical index data
    
    ind_df = fetch_index_historial(index_symbol='NSE:NIFTY 50' , lookback_days=lookback , bar_interval=30)
    
    #=== now we do ranking and other stuff based on previous day data
   
    
    # Function wrapper to pass multiple arguments to map
    def process_symbol(symbol):
        return fetch_and_rank_data(symbol, lookback=lookback, time_interval=30, indexdf=ind_df, index_req=1)
        
           
    # Use ThreadPoolExecutor to process symbols in parallel
    num_cores = os.cpu_count()
    with ThreadPoolExecutor(max_workers=num_cores) as executor:
        results = list(executor.map(process_symbol, stocks_to_process))
    
    
    combined_ranks_df = pd.concat(results , axis=1 , join='outer')
    
    #print(combined_ranks_df.tail(5)) 
        
    rank_number = stock_limit

        
    comb_ranks = combined_ranks_df.rank(ascending=False ,axis=1)
    
    
    last_row = comb_ranks.iloc[-1]
    #print(last_row)
    today_stocks = last_row[last_row <= rank_number].index.tolist()
    #
    
    
    today_stocks = [stocks.split('_')[0] for stocks in today_stocks]    
    print( "Selected stocks are : ", today_stocks)
    
    return today_stocks , ind_df        
        
    
    

In [ ]:
today_stocks , ind_df = pre_strategy_code(stock_limit=20 , lookback=30)

In [7]:
import asyncio
import aiohttp

async def fetch_live_ltp_async(symbol):
    async with aiohttp.ClientSession() as session:
        encoded_symbol = urllib.parse.quote(symbol)
        url = f"http://65.0.126.220:8002/api/ltp?instrument={encoded_symbol}"
        async with session.get(url) as response:
            response.raise_for_status()
            data = await response.json()
            return data.get('data')

async def fetch_all_ltp_async(stock_list):
    tasks = [fetch_live_ltp_async(sym) for sym in stock_list]
    results = await asyncio.gather(*tasks)
    return dict(zip(stock_list, results))

# Assuming stock_list is defined
# Use await directly in a Jupyter Notebook cell
# results = await fetch_all_ltp_async(today_stocks)
# print(results)


In [8]:
import asyncio
import aiohttp
import pandas as pd


# Define an asynchronous function to fetch live candle data for a given symbol.
async def fetch_live_candles_async(symbol, lookback_days=0, product='stocks', interval=30):
    # URL-encode the symbol to handle special characters
    encoded_symbol = urllib.parse.quote(symbol)
    
    # Calculate the start and end dates for the data fetch
    end_date = datetime.today().date()
    start_date = end_date - timedelta(days=lookback_days)
    start_date_str = start_date.strftime('%Y-%m-%d')
    end_date_str = end_date.strftime('%Y-%m-%d')

    # Construct the URL for the API request
    url = f"http://65.0.126.220:8002/api/{product}/candles?interval={interval}&instrument={encoded_symbol}&from={start_date_str}&to={end_date_str}"

    # Use aiohttp to make the asynchronous GET request
    async with aiohttp.ClientSession() as session:
        async with session.get(url) as response:
            # Automatically raise an exception for 4XX/5XX responses
            response.raise_for_status()
            # Parse the JSON response
            data_json = await response.json()
            if 'data' in data_json :
                # Convert the 'data' list to a DataFrame
                df = pd.DataFrame(data_json['data'])
                # Convert the 'Timestamp' column to datetime and set it as the DataFrame index
                df['Timestamp'] = pd.to_datetime(df['Timestamp'])
                df.set_index('Timestamp', inplace=True)
                return df
            else:
                # Log and return an empty DataFrame if no data is found
                print(f"No data found for {symbol}.")
                return pd.DataFrame()

# Asynchronously fetch live candles for a list of symbols
async def fetch_all_live_candles_async(stock_list, lookback_days=10, product='stocks', interval=30):
    # Create a list of tasks for each symbol
    tasks = [fetch_live_candles_async(symbol=sym, lookback_days=lookback_days, product=product, interval=interval) for sym in stock_list]
    # Gather all tasks to run concurrently, waiting for all to complete
    
    results = await asyncio.gather(*tasks)
    # Combine results into a dictionary {symbol: DataFrame}
    return {sym: result for sym, result in zip(stock_list, results)}

# # Example usage
# async def main():
#     stock_list = ['NSE:RELIANCE', 'NSE:TCS']  # Example stock symbols
#     all_candles = await fetch_all_live_candles_async(stock_list=stock_list, lookback_days=0, product='stocks', interval=30)
#     for symbol, candles in all_candles.items():
#         print(f"{symbol} candles:\n{candles}")

# # Run the main function
# await main()


In [ ]:
all_candles = await fetch_all_live_candles_async(stock_list=today_stocks, lookback_days=0, product='stocks', interval=30)

In [ ]:
today_2 = ['NSE:ACC' , 'NSE:IDEA' , 'NSE:HDFCBANK']
results = await fetch_all_ltp_async(today_2)
print(results)


In [ ]:
for stocks in today_stocks:
    candles = fetch_live_candles(symbol=stocks , lookback_days=2 , product='stocks' , interval=30)
    print(candles)

In [9]:

import pytz
from datetime import datetime, timedelta , time  # Alias datetime.time
import time as timet   # This allows you to use time.sleep()


def calculate_run_times(start_hour, start_minute, end_hour, end_minute, interval):
    """
    Calculate all run times within a specified window and interval.
    
    Parameters:
    - start_hour (int): Hour when the window starts (24-hour format).
    - start_minute (int): Minute when the window starts.
    - end_hour (int): Hour when the window ends (24-hour format).
    - end_minute (int): Minute when the window ends.
    - interval (int): Interval between each run time, in minutes.
    
    Returns:
    - run_times (list of datetime): List of calculated run times.
    """
    india_tz = pytz.timezone('Asia/Kolkata')
    today = datetime.now(india_tz).date()
    start_time = datetime.combine(today, time(start_hour, start_minute), tzinfo=india_tz) + timedelta(minutes=interval)
    end_time = datetime.combine(today, time(end_hour, end_minute), tzinfo=india_tz)
    
    run_times = []
    current_time = start_time
    while current_time <= end_time:
        run_times.append(current_time)
        current_time += timedelta(minutes=interval)
    
    run_times = [tm.time() for tm in run_times]
    
    return run_times


runt_times = calculate_run_times(start_hour=9 , start_minute=15 , end_hour=15 , end_minute=25 , interval=2)

In [10]:

# Use this wrapper function to call your async function from synchronous code
def fetch_all_live_candles_sync(stock_list, lookback_days, product, interval):
    # This function uses asyncio.run to run the async function from synchronous code
    try:
        result = asyncio.run(fetch_all_live_candles_async(stock_list, lookback_days, product, interval))
        return result
    except RuntimeError as e:
        print(f"Error running async function: {e}")
        return {}

In [ ]:
#fetch_all_live_candles_sync(today_stocks , 0 , 'stocks' , interval=30)

results = await fetch_all_live_candles_async(today_stocks , 0 , product='stocks' , interval=30)

In [11]:
import pandas as pd
import concurrent.futures

# Assuming `today_stocks_his` and `results` are your dictionaries of DataFrames
#com_df = {key: pd.concat([today_stocks_his[key], results[key]], axis=0) for key in today_stocks_his if key in results}


def process_symbol_concurrent(com_df, index):
    with concurrent.futures.ProcessPoolExecutor() as executor:
        # Submit all tasks
        futures = [executor.submit(process_symbol2, df, index) for df in com_df.values()]
        
        results = []
        for future in concurrent.futures.as_completed(futures):
            try:
                # Attempt to get the result with a timeout
                result = future.result(timeout=60)  # 60 seconds timeout
                results.append(result)
            except concurrent.futures.TimeoutError:
                print('A task exceeded the time limit and was not completed.')
            except Exception as exc:
                print(f'A task generated an exception: {exc}')
    return results


# # Example usage
# index = ind_com
# ss2 = process_dataframes_concurrent(concatenated_dfs, index=ind_com)


def sort_dataframes_dict(dataframes_dict):
    
    #--- function used to remove duplicates and than sort by index ( if timestamp)
    for symbol, df in dataframes_dict.items():
        if not df.index.is_monotonic_increasing or df.index.duplicated().any():
            df.sort_index(inplace=True)
            df = df.loc[~df.index.duplicated(keep='first')]
            dataframes_dict[symbol] = df
    return dataframes_dict


In [12]:
import pytz
import asyncio

strategy_start = time(2,25) 
strategy_end = time(15,25)

india_tz = pytz.timezone('Asia/Kolkata')
india_time = datetime.now(india_tz)
index_symbol = "NSE:NIFTY 50"

code_run_before_start = False 

lookback_days = 30

strategy_interval = 2


runt_times = calculate_run_times(start_hour=strategy_start.hour , start_minute=strategy_start.minute , end_hour=strategy_end.hour , end_minute=strategy_end.minute , interval=strategy_interval)

print("\n run times are " , runt_times)

while True:
    
        now = datetime.now(india_tz)
        timet.sleep(0.1)
        #print("\n Current time: " , now)
        

        if strategy_start<now.time()<strategy_end:
                
                
                #print("Strategy is live " , )
                now_rounded = now.replace(second=0, microsecond=0).time()
                print(now_rounded)
                
                next_run = get_next_run_time(start_hour=strategy_start.hour , start_minute=strategy_start.minute , interval=1)        
                
                if now_rounded in runt_times:
                        
                        # Place your code here that you want to run at each scheduled time
                        print(f"Running code for scheduled time: {now_rounded}")
                        
                        #== we fetch live index candles and combine with historical index data                                                
                        ind_live = fetch_live_candles(symbol=index_symbol ,lookback_days=0 , interval=30 , product='indexes')
                        
                        #print( "Index live candles", ind_live.tail(2) )
                        
                        if ind_live is not None and not ind_live.empty:
                                ind_com = pd.concat([ind_df, ind_live], join='outer', axis=0)
                        
                        else:
                                
                                
                                ind_com =ind_df
                                print("\n Error fetching live index candle")        
                        
                        
                        print("Historical index is " , ind_df.tail(5))
                        ind_com = ind_com[~ind_com.index.duplicated(keep='first')]
                        
                        print("\n Merged today index with yesterday to create complete index : \n" , ind_com )

                        
                        ind_com = process_index(ind_com)
                        
                        
                        #--- now we fetch live Stock candles
                        try:
                                results = await fetch_all_live_candles_async(today_stocks , 0 , product='stocks' , interval=30)
                                
                                #print("Live stock candles:", results.tail(5))
                                await asyncio.sleep(0.1)
                                #print(results)
                                
                        except RuntimeError as error:
                                print(f"Error: {error}")
                        
                        
                        
                        
                        # Remove the executed time from the list
                        if (now_rounded) in runt_times:
                                print("\n Removing run time: " ,now_rounded )
                                runt_times.remove(now_rounded)
                                                        
                        
                        
                        #=== This is the dataframe on which the strategy would run
                        
                        com_df =  {key: pd.concat([today_stocks_his[key], results[key]], axis=0)  for key in today_stocks_his if key in results}
                        
                        #== sort by index and remove duplicates
                        com_df = sort_dataframes_dict(com_df)
                        
                        print("\n After removing duplicates and sorting , stock_data is \n " , com_df)
                        
                        #-- process_symbol for all symbols for today
                        com_dfs = process_symbol_concurrent(com_df, index=ind_com)
                        
                        
                        
                        
                        
                        # Sleep for slightly longer than the interval to ensure we don't hit the same time twice
                        # This is to account for any execution time of your code and minor discrepancies
                        timet.sleep(strategy_interval * 30 )
                        
                        break 
                       
                                
                
        elif now.time() <= strategy_start and not code_run_before_start:
                
                print("Before strategy start time, waiting for market to open.\n Doing preprocessing")
                
                
                today_stocks , ind_df = pre_strategy_code(stock_limit=40 , lookback=lookback_days)
                        # Your one-time code here
                print("\n Stocks for today are : \n" , today_stocks)        
                timet.sleep(5)
                        # Set the flag to True to prevent this block from running again
                
                #=== this should run before market starts
                
                #today_stocks_his = await fetch_all_live_candles_async(today_stocks , lookback_days=lookback_days , product='stocks' , interval=30)
                
                
                today_stocks_his = await fetch_historical_data_from_date_for_symbols(symbols=today_stocks , time_interval=30 , start_date=(now.date() - timedelta(days=lookback_days)) , end_date=(now.date()-timedelta(days=1)))
                
                #print(today_stocks_his)
                        
                code_run_before_start = True
                
                 
        
               
        elif now.time() >= strategy_end:
                print("\n After strategy end time, market closed.")
                timet.sleep(10)
                break


 run times are  [datetime.time(2, 27), datetime.time(2, 29), datetime.time(2, 31), datetime.time(2, 33), datetime.time(2, 35), datetime.time(2, 37), datetime.time(2, 39), datetime.time(2, 41), datetime.time(2, 43), datetime.time(2, 45), datetime.time(2, 47), datetime.time(2, 49), datetime.time(2, 51), datetime.time(2, 53), datetime.time(2, 55), datetime.time(2, 57), datetime.time(2, 59), datetime.time(3, 1), datetime.time(3, 3), datetime.time(3, 5), datetime.time(3, 7), datetime.time(3, 9), datetime.time(3, 11), datetime.time(3, 13), datetime.time(3, 15), datetime.time(3, 17), datetime.time(3, 19), datetime.time(3, 21), datetime.time(3, 23), datetime.time(3, 25), datetime.time(3, 27), datetime.time(3, 29), datetime.time(3, 31), datetime.time(3, 33), datetime.time(3, 35), datetime.time(3, 37), datetime.time(3, 39), datetime.time(3, 41), datetime.time(3, 43), datetime.time(3, 45), datetime.time(3, 47), datetime.time(3, 49), datetime.time(3, 51), datetime.time(3, 53), datetime.time(3, 55

In [ ]:
ind_live

In [ ]:
dd = await fetch_all_live_candles_async(stock_list=today_stocks, lookback_days=0, product='stocks', interval=30)

In [ ]:
for symbol, df in com_df.items():
    # Sort the DataFrame by its index if it's not already sorted
    if not df.index.is_monotonic_increasing:
        df.sort_index(inplace=True)
    
    # Remove duplicates based on the index
    df = df.loc[~df.index.duplicated(keep='first')]
    
    # Update the dictionary with the cleaned DataFrame
    com_df[symbol] = df

In [ ]:
def sort_dataframes_dict(dataframes_dict):
    
    #--- function used to remove duplicates and than sort by index ( if timestamp)
    for symbol, df in dataframes_dict.items():
        if not df.index.is_monotonic_increasing or df.index.duplicated().any():
            df.sort_index(inplace=True)
            df = df.loc[~df.index.duplicated(keep='first')]
            dataframes_dict[symbol] = df





In [ ]:
concatenated_dfs = {key: pd.concat([today_stocks_his[key], results[key]], axis=0)
                    for key in today_stocks_his if key in results}

In [ ]:
import pandas as pd
import concurrent.futures

# Assuming `today_stocks_his` and `results` are your dictionaries of DataFrames
#com_df = {key: pd.concat([today_stocks_his[key], results[key]], axis=0) for key in today_stocks_his if key in results}


def process_dataframes_async(com_df, index):
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = [executor.submit(process_symbol2, df, index) for df in com_df.values()]
        # Wait for all futures to complete
        concurrent.futures.wait(futures)
        # Collect results
        results = [future.result() for future in futures]
    return results

# Example usage
index = ind_com
ss = process_dataframes_async(concatenated_dfs, index=ind_com)


In [ ]:
def compare_dataframe_lists(list1, list2):
    # Check if the lists are the same length
    if len(list1) != len(list2):
        return False
    
    # Iterate through both lists and compare DataFrames
    for df1, df2 in zip(list1, list2):
        if not df1.equals(df2):
            return False  # Found DataFrames that are not equal
    return True  # All corresponding DataFrames are equal

In [27]:
from concurrent.futures import ProcessPoolExecutor, as_completed
import os


def process_wrapper(df):
    #symbol, df = symbol_df_tuple
    return process_signal(df, side=1, intra_Exit=0, n_day_Exit=2)

# Using ProcessPoolExecutor to process each DataFrame in parallel
with ProcessPoolExecutor() as executor:
    # Create a list of future tasks
    futures = [executor.submit(process_wrapper, item) for item in com_dfs]

    # As each future completes, print the result or error
    for future in as_completed(futures):
        try:
            symbol, result_df = future.result()
            print(f"Processing complete ")
        except Exception as e:
            print(f"Error processing : {e}")



 Processing signals of : 
 Processing signals of : 
 Processing signals of :  
 Processing signals of : 
 Processing signals of :  NSE:AMBUJACEM
 Processing signals of :  
 Processing signals of :  
 
 Processing signals of : NSE:ABBOTINDIANSE:ADANIENT NSE:BAJAJFINSV
 Processing signals of :  NSE:BAJFINANCE
 Processing signals of : 

 

 Processing signals of : NSE:ADANIPORTS
 Processing signals of : NSE:BIOCON
  
 Processing signals of : NSE:DLF

 Processing signals of :  NSE:CHOLAFINNSE:GRASIM


 Processing signals of :   
 Processing signals of : 

NSE:DRREDDYNSE:HEROMOTOCO  
 Processing signals of :  

/tmp/ipykernel_21521/3164365030.py:66: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if ( df['signal'][i-1]==1 or df['signal'][i]==1) and position == 0  and df['Close'][i]<df['Close'][i-1] and df['indClose'][i]>df['indtodayo'][i]*0.999 :



 Processing signals of : NSE:BHEL
NSE:ASHOKLEY
 Processing signals of :  

/tmp/ipykernel_21521/3164365030.py:66: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if ( df['signal'][i-1]==1 or df['signal'][i]==1) and position == 0  and df['Close'][i]<df['Close'][i-1] and df['indClose'][i]>df['indtodayo'][i]*0.999 :


NSE:GODREJCP

...

/tmp/ipykernel_21521/1375693250.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['trade_end'] = (df[bar_count_col].shift(-1) < df[bar_count_col]) | (df[bar_count_col].shift(-1) == 0) | df[bar_count_col].eq(0)


 

-- entry Price - :  -- entry Price - :  R1 today  273.6  

/tmp/ipykernel_21521/1375693250.py:34: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['trade_start'] = df['trade_end'].shift(1).fillna(False) | df[bar_count_col].eq(1)


 position at  3705.0 1140.0150.0

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [29]:
import pandas as pd


def create_tradelog2(df, symbol):
    
    print("\nCreating trade log for", symbol)
    
    if df.empty:
        print("DataFrame is empty. No data to process.")
        return pd.DataFrame()
    df['position_change'] = df[f'{symbol}_position'].diff()
    
        # Construct column names dynamically
    columns = [
        f'{symbol}bar_count', f'{symbol}_position',  'position_change' , f'{symbol}_ret', f'{symbol}_Close' , f'{symbol}_order']

    # Now filter the DataFrame to include only the columns for this symbol
    df = df[columns]

    
    
    # Assuming 'bar_count_col' and other columns are defined as per your data
    bar_count_col = f'{symbol}bar_count'

    # Check if the essential column exists in the DataFrame
    if bar_count_col not in df.columns:
        print(f"Missing essential column: {bar_count_col}")
        return pd.DataFrame()
    
    # Identifying the end of a trade
    df['trade_end'] = (df[bar_count_col].shift(-1) < df[bar_count_col]) | (df[bar_count_col].shift(-1) == 0) | df[bar_count_col].eq(0)

    # Creating 'trade_start' to identify the start of a trade
    df['trade_start'] = df['trade_end'].shift(1).fillna(False) | df[bar_count_col].eq(1)
    df['trade_start'] = df['trade_start'].astype(int)

    # Generating 'trade_id' for each trade
    df['trade_id'] = df['trade_start'].cumsum()

    # Filtering out rows not part of any trade (i.e., before any trade has started)
    df = df[df['trade_id'] > 0]
    
    # Processing each trade to gather trade details
    trades_info = []
    for trade_id, trade_group in df.groupby('trade_id'):
        if trade_group.empty or trade_group[bar_count_col].max() == 0:
            continue  # Skip empty trades or trades without bar counts
        
        #print("\n Individual trade found as " , trade_group)
        
        #print("\n Exit price: " , trade_group.iloc[-1][f'{symbol}_Close'])
        
        positionn = trade_group[f'{symbol}_position'].iloc[0]
        side = 'long' if positionn == 1 else 'Short' if positionn == -1 else 'flat'
        trade_details = {
            'Symbol': symbol,
            'Side': side,
            'Trade ID': trade_id,
            'Start Time': trade_group.index[0],
            'End Time': trade_group.index[-1],
            'Entry Price': trade_group.iloc[0][f'{symbol}_Close'],  # Assuming 'Close' is the close price column
            'Exit Price': trade_group.iloc[-1][f'{symbol}_Close'],  # Modify as per your DataFrame's column names
            
            
            
            # Include other details as needed
        }
        trades_info.append(trade_details)
    
    # Converting the trades information into a DataFrame
    trade_summary_df = pd.DataFrame(trades_info)
    return trade_summary_df

# Replace 'your_symbol' and 'Close' with actual column names as per your DataFrame
# Ensure your DataFrame 'df' is indexed appropriately (e.g., datetime index) and contains the necessary columns


In [23]:
print(com_dfs[2].index[0])
print(com_dfs[2].index[-1])


2024-03-06 09:15:00+05:30
2024-04-04 15:15:00+05:30


In [30]:
process_signal(com_dfs[2], side=1, intra_Exit=0, n_day_Exit=2)


 Processing signals of :  NSE:ADANIENT

 Position taken long at :  2024-03-21 11:15:00+05:30 -- entry Price - :   3093.4 R1 today 3087.9970000000003

 Carryforward exit EOD long at 2024-03-26 10:15:00+05:30 1 position at   3134.0 Day_count:  2

 Position taken long at :  2024-03-28 10:45:00+05:30 -- entry Price - :   3154.55 R1 today 3145.9961000000003

 Resistance target   2024-03-28 14:45:00+05:30 at   3181.25  R2 prev:  3155.1833333333334  entryprice 3154.55
[(Timestamp('2024-03-21 11:15:00+0530', tz='UTC+05:30'), 'NSE:ADANIENT', 1, 3093.4, 1), (Timestamp('2024-03-26 10:15:00+0530', tz='UTC+05:30'), 'NSE:ADANIENT', 0, 3134.0, 16), (Timestamp('2024-03-28 10:45:00+0530', tz='UTC+05:30'), 'NSE:ADANIENT', 1, 3154.55, 1), (Timestamp('2024-03-28 14:45:00+0530', tz='UTC+05:30'), 'NSE:ADANIENT', 0, 3181.25, 20)]

Creating trade log for NSE:ADANIENT

 
 The Outputting to trade file  /home/aakash_tandon/Python_strategies/Live_aakash_fno_CF_v1/symbol_wise_trades/NSE:ADANIENT.csv

 
 The Outpu

/tmp/ipykernel_21521/3164365030.py:66: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if ( df['signal'][i-1]==1 or df['signal'][i]==1) and position == 0  and df['Close'][i]<df['Close'][i-1] and df['indClose'][i]>df['indtodayo'][i]*0.999 :
/tmp/ipykernel_21521/3164365030.py:73: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print("\n Position taken long at : " , df.index[i] , "-- entry Price - :  " , entry_price , 'R1 today' ,df['R1'][i]*1.002  )
/tmp/ipykernel_21521/3164365030.py:85: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as l

,NSE:ADANIENT_Close,NSE:ADANIENT_position,NSE:ADANIENT_ret,NSE:ADANIENTbar_count,NSE:ADANIENT_order
Timestamp,,,,,
2024-03-06 09:15:00+05:30,3282.00,0,0.0,0,0
2024-03-06 09:45:00+05:30,3240.20,0,0.0,0,0
2024-03-06 10:15:00+05:30,3259.05,0,0.0,0,0
2024-03-06 10:45:00+05:30,3240.10,0,0.0,0,0
2024-03-06 11:15:00+05:30,3216.60,0,0.0,0,0
...,...,...,...,...,...
2024-04-04 13:15:00+05:30,3236.30,0,0.0,0,0
2024-04-04 13:45:00+05:30,3228.95,0,0.0,0,0
2024-04-04 14:15:00+05:30,3230.25,0,0.0,0,0


In [ ]:
com_dfs[0].columns

In [ ]:
df[['NSE:ADANIENTbar_count', 'NSE:ADANIENT_order', 'NSE:ADANIENT_position',
       'NSE:ADANIENT_ret', 'NSE:ADANIENT_Close', 'position_change']]

In [ ]:


"""ATR



 Processing signals of :  NSE:ABB

 Position taken long at :  2024-03-18 14:15:00+05:30 -- entry Price - :   5746.85 R1 today 5741.426599999999
 Fixed stoploss:  S1 is :  5631.9130000000005  -- exit  at 5631.9130000000005 time is  14:45:00 Low of bar is : 5611.5

 Position taken long at :  2024-03-20 13:45:00+05:30 -- entry Price - :   5743.35 R1 today 5759.329000000001

 Carryforward exit EOD long at 2024-03-22 09:15:00+05:30 1 position at   5851.0 Day_count:  2

 Position taken long at :  2024-03-28 12:15:00+05:30 -- entry Price - :   6426.65 R1 today 6398.705200000001

 Carryforward exit EOD long at 2024-04-02 09:15:00+05:30 1 position at   6430.3 Day_count:  2




"""

df = pd.read_csv(r'/home/aakash_tandon/Python_strategies/Live_aakash_fno_CF_v1/symbol_wise_dataframe/NSE:ABB.csv')

df['Timestamp'] = pd.to_datetime(df['Timestamp'])
df = df.set_index('Timestamp')

symbol = df['Symbol'].iloc[0]

print(symbol)
# Construct column names dynamically
columns = [
    f'{symbol}bar_count', f'{symbol}_order', f'{symbol}_position',f'{symbol}_ret', f'{symbol}_Close', 'position_change']

# Now filter the DataFrame to include only the columns for this symbol
df = df[columns]


# Initialize a new trade ID for grouping
df['trade_id'] = (df[f'{symbol}bar_count'] == 0).cumsum()

# Filter out rows not part of any trade (bar_count == 0)
trading_periods = df[df[f'{symbol}bar_count'] >= 1]

# Initialize lists to collect entry and exit prices for trades
entry_prices = []
exit_prices = []

# Iterate over each trade group
for trade_id, group in trading_periods.groupby('trade_id'):
    if not group.empty:
        entry_price = group.iloc[0][f'{symbol}_Close']  # First element's close price
        exit_price = group.iloc[-1][f'{symbol}_Close']  # Last element's close price
        
        # Append to lists
        entry_prices.append(entry_price)
        exit_prices.append(exit_price)

# Display results
print("Entry Prices:", entry_prices)
print("Exit Prices:", exit_prices)




In [36]:
#----- function to read trade logs and find total profit from all of symbols for them

def read_trade_log(logloc):
    
    
    
    symbol = os.path.splitext(os.path.basename(logloc))[0].lstrip('_')
    print("\n \n Processing trade log of symbol" , symbol)
    
    
    df = pd.read_csv(logloc)
    if df.empty:
        print("No trades found")
        return None
    else:
        print("\Columns are: " , df.columns)
        df['Entry Price'] = df['Entry Price'].astype('float')
        df['Exit Price'] = df['Exit Price'].astype('float')
        
        # Drop rows where entry_price or exit_price is NaN
        df.dropna(subset=['Entry Price', 'Exit Price'], inplace=True)
        
        
        if 'Side' in df.columns:
            df['returns'] = np.where(df['Side']=='long' , (df['Exit Price']/df['Entry Price'])-1 , (df['Exit Price']-df['Entry Price'])/df['Entry Price'])
        #print(df)
        
        
        net_return = (100000*df["returns"]).cumsum()
        net_return.dropna(inplace=True)
        print( "Symbol return from trade log is : ", net_return.iloc[-1]) 
        return symbol , net_return.iloc[-1]


    
def concatenate_trades(logloc , comb_df , add_slippages=0 , slippage_in_ticks=1 ,min_movement = 0.05):
    
    
    symbol = os.path.splitext(os.path.basename(logloc))[0].lstrip('_')
    print("\n Concqatenating trades of symbol: " , symbol)
    print("\n Slippage used: (in ticks)" , slippage_in_ticks)
    
    
    df = pd.read_csv(logloc)
    
    print("\Columns are: " , df.columns)
    if df.empty:
        print("No trades found")
        return None
    else:
        
        # Convert entry_price and exit_price to numeric, set errors='coerce' to handle non-numeric
        df['Entry Price'] = pd.to_numeric(df['Entry Price'], errors='coerce')
        df['Exit Price'] = pd.to_numeric(df['Exit Price'], errors='coerce')
  
        
        
        if add_slippages>0:


            df['Entry Price'] = np.where(df['Side']=='long' , (df['Entry Price'] + (slippage_in_ticks*min_movement))  ,  (df['Entry Price'] - (slippage_in_ticks*min_movement)))

            df['Exit Price'] = np.where(df['Side']=='long' , (df['Exit Price'] - (slippage_in_ticks*min_movement))  ,  (df['Exit Price'] + (slippage_in_ticks*min_movement)))

            #print("\n After adding slippages" , df)

               
        # Drop rows where entry_price or exit_price is NaN
        df.dropna(subset=['Entry Price', 'Exit Price'], inplace=True)
        
        comb_df = pd.concat([comb_df , df] , axis=0 , join='outer')
        
        
        return comb_df
    
    
    
    
cdf = pd.DataFrame()
all_trades = pd.DataFrame()

log_path = trades_strategy_folder


print("\ Reading all trade files from folder " , log_path)

if os.path.exists(log_path):
        file_paths = glob.glob(log_path + "/*.csv")
        
        for file in file_paths:
            
            result = read_trade_log(file)
            if result is not None:
                sym, profit = result
            else:
                print("\n No trade returned from sym: " , sym)
            
            print("\n The trade returns dataframe is : " , cdf)
            
            cdf = cdf.append({'Symbol': sym, 'Profit': profit}, ignore_index=True)
            all_trades = concatenate_trades(file , all_trades , add_slippages=1 ,slippage_in_ticks=2 )
        
        all_trades.to_csv(output_all_trades_file)

        
        

        

#-- this is the win ratio
print(len(all_trades[all_trades['exit_price']>all_trades['entry_price']])/len(all_trades))

# #== Code to read all the trades from the strategy folder and than find the trade stats

# if os.path.exists(output_all_trades_file):
#     print("Output htlml trade file exists as: " ,output_all_trades_file)
    
#     create_trade_report(output_all_trades_file , starting_capital=10000000 , output_html_path=trade_html_file)        
        

\ Reading all trade files from folder  /home/aakash_tandon/Python_strategies/Live_aakash_fno_CF_v1/symbol_wise_trades

 
 Processing trade log of symbol NSE:ADANIENT
\Columns are:  Index(['Unnamed: 0', 'Symbol', 'Side', 'Trade ID', 'Start Time', 'End Time',
       'Entry Price', 'Exit Price'],
      dtype='object')
Symbol return from trade log is :  2158.8681889047166

 The trade returns dataframe is :  Empty DataFrame
Columns: []
Index: []


AttributeError: 'DataFrame' object has no attribute 'append'

In [ ]:
# Filter columns for position and return
position_columns = [col for col in combined_df.columns if 'position' in col]
return_columns = [col for col in combined_df.columns if '_ret' in col]

# Calculate the sum of returns for assets where position is non-zero
combined_df['sum_of_rturns'] = combined_df[return_columns].sum(axis=1)

# Count the number of non-zero positions for each row
combined_df['non_zero_positions'] = combined_df[position_columns].apply(lambda x: (x!=0).sum(), axis=1)

symbol_Equity = (combined_df[return_columns]*100000)

total_profit_from_returns = symbol_Equity.cumsum().iloc[-1]

Portfolio_EC = len(return_columns)*100000+(symbol_Equity.sum(axis=1).cumsum())

Portfolio_EC.plot()

In [ ]:
3093.4*(1+df['NSE:ADANIENT_ret'].sum())

In [ ]:
(1 + df['NSE:ADANIENT_ret']).cumprod() - 1

In [ ]:
df['NSE:ADANIENT_Close']

In [ ]:
start_price = df["NSE:ADANIENT_Close"].iloc[0]
end_price = df["NSE:ADANIENT_Close"].iloc[-1]
trade_return = end_price / start_price - 1

In [ ]:
trade_return

In [ ]:
df

In [ ]:
df